## Explora la agrupación K-Means usando R y los principios de datos ordenados.

### [**Cuestionario previo a la lección**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

En esta lección, aprenderás cómo crear grupos utilizando el paquete Tidymodels y otros paquetes del ecosistema de R (los llamaremos amigos 🧑‍🤝‍🧑), y el conjunto de datos de música nigeriana que importaste anteriormente. Cubriremos los conceptos básicos de K-Means para la agrupación. Ten en cuenta que, como aprendiste en la lección anterior, hay muchas formas de trabajar con agrupaciones y el método que utilices depende de tus datos. Intentaremos K-Means ya que es la técnica de agrupación más común. ¡Comencemos!

Términos que aprenderás:

-   Puntuación de silueta

-   Método del codo

-   Inercia

-   Varianza

### **Introducción**

[La agrupación K-Means](https://wikipedia.org/wiki/K-means_clustering) es un método derivado del dominio del procesamiento de señales. Se utiliza para dividir y particionar grupos de datos en `k grupos` basados en similitudes en sus características.

Los grupos pueden visualizarse como [diagramas de Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), que incluyen un punto (o 'semilla') y su región correspondiente.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografía por Jen Looper</figcaption>


La agrupación K-Means tiene los siguientes pasos:

1.  El científico de datos comienza especificando el número deseado de grupos a crear.

2.  Luego, el algoritmo selecciona aleatoriamente K observaciones del conjunto de datos para servir como los centros iniciales de los grupos (es decir, los centroides).

3.  A continuación, cada una de las observaciones restantes se asigna a su centroide más cercano.

4.  Después, se calcula el nuevo promedio de cada grupo y el centroide se mueve al promedio.

5.  Ahora que los centros han sido recalculados, cada observación se verifica nuevamente para ver si podría estar más cerca de un grupo diferente. Todos los objetos se reasignan nuevamente utilizando los promedios actualizados de los grupos. Los pasos de asignación de grupos y actualización de centroides se repiten iterativamente hasta que las asignaciones de grupos dejan de cambiar (es decir, cuando se logra la convergencia). Por lo general, el algoritmo termina cuando cada nueva iteración resulta en un movimiento insignificante de los centroides y los grupos se vuelven estáticos.

<div>

> Ten en cuenta que debido a la aleatoriedad de las observaciones iniciales k utilizadas como centroides iniciales, podemos obtener resultados ligeramente diferentes cada vez que aplicamos el procedimiento. Por esta razón, la mayoría de los algoritmos utilizan varios *inicios aleatorios* y eligen la iteración con el menor WCSS. Por lo tanto, se recomienda encarecidamente ejecutar K-Means con varios valores de *nstart* para evitar un *óptimo local no deseado.*

</div>

Esta breve animación utilizando la [obra](https://github.com/allisonhorst/stats-illustrations) de Allison Horst explica el proceso de agrupación:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Obra de @allison_horst</figcaption>



Una pregunta fundamental que surge en la agrupación es esta: ¿cómo sabes cuántos grupos separar en tus datos? Una desventaja de usar K-Means incluye el hecho de que necesitarás establecer `k`, es decir, el número de `centroides`. Afortunadamente, el `método del codo` ayuda a estimar un buen valor inicial para `k`. Lo probarás en un momento.

### 

**Requisito previo**

Continuaremos justo desde donde lo dejamos en la [lección anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), donde analizamos el conjunto de datos, hicimos muchas visualizaciones y filtramos el conjunto de datos a observaciones de interés. ¡Asegúrate de revisarlo!

Necesitaremos algunos paquetes para completar este módulo. Puedes instalarlos como: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativamente, el script a continuación verifica si tienes los paquetes necesarios para completar este módulo y los instala por ti en caso de que falten algunos.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


¡Vamos a ponernos en marcha!

## 1. Un baile con los datos: Reduce a los 3 géneros musicales más populares

Este es un repaso de lo que hicimos en la lección anterior. ¡Vamos a analizar y desglosar algunos datos!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 ¡Eso salió bien!

## 2. Más exploración de datos.

¿Qué tan limpios están estos datos? Vamos a verificar la presencia de valores atípicos utilizando diagramas de caja. Nos concentraremos en las columnas numéricas con menos valores atípicos (aunque podrías limpiar los valores atípicos). Los diagramas de caja pueden mostrar el rango de los datos y ayudar a elegir qué columnas usar. Nota: los diagramas de caja no muestran la varianza, un elemento importante para datos que se puedan agrupar bien. Por favor, consulta [esta discusión](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) para más información.

Los [diagramas de caja](https://es.wikipedia.org/wiki/Diagrama_de_caja) se utilizan para representar gráficamente la distribución de datos `numéricos`, así que comencemos *seleccionando* todas las columnas numéricas junto con los géneros musicales populares.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


¿Ves cómo el selector `where` facilita esto 💁? Explora otras funciones similares [aquí](https://tidyselect.r-lib.org/).

Como vamos a crear un diagrama de caja para cada característica numérica y queremos evitar usar bucles, reformateemos nuestros datos a un formato *más largo* que nos permita aprovechar los `facets`, es decir, subgráficos que muestran un subconjunto de los datos cada uno.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


¡Mucho más largo! ¡Ahora es momento de algunos `ggplots`! Entonces, ¿qué `geom` usaremos?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


¡Fácil-gg!

Ahora podemos ver que estos datos están un poco desordenados: al observar cada columna como un diagrama de caja, puedes notar valores atípicos. Podrías revisar el conjunto de datos y eliminar estos valores atípicos, pero eso haría que los datos fueran bastante mínimos.

Por ahora, elijamos qué columnas utilizaremos para nuestro ejercicio de agrupamiento. Seleccionemos las columnas numéricas con rangos similares. Podríamos codificar `artist_top_genre` como numérico, pero por ahora lo descartaremos.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Cálculo de agrupamiento k-means en R

Podemos calcular k-means en R utilizando la función incorporada `kmeans`. Consulta `help("kmeans()")`. La función `kmeans()` acepta un marco de datos con todas las columnas numéricas como su argumento principal.

El primer paso al usar el agrupamiento k-means es especificar el número de clústeres (k) que se generarán en la solución final. Sabemos que hay 3 géneros musicales que extrajimos del conjunto de datos, así que probemos con 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


El objeto kmeans contiene varios elementos de información que están bien explicados en `help("kmeans()")`. Por ahora, enfoquémonos en algunos. Vemos que los datos se han agrupado en 3 clusters de tamaños 65, 110, 111. El resultado también incluye los centros de los clusters (medias) para los 3 grupos a través de las 5 variables.

El vector de agrupamiento es la asignación de cluster para cada observación. Usemos la función `augment` para agregar la asignación de cluster al conjunto de datos original.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfecto, hemos dividido nuestro conjunto de datos en un conjunto de 3 grupos. Entonces, ¿qué tan bueno es nuestro agrupamiento 🤷? Echemos un vistazo al `Silhouette score`.

### **Silhouette score**

El [análisis de Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) se puede utilizar para estudiar la distancia de separación entre los clústeres resultantes. Este puntaje varía de -1 a 1, y si el puntaje está cerca de 1, el clúster es denso y está bien separado de otros clústeres. Un valor cercano a 0 representa clústeres superpuestos con muestras muy cercanas al límite de decisión de los clústeres vecinos. [fuente](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

El método de Silhouette promedio calcula el promedio de Silhouette de las observaciones para diferentes valores de *k*. Un puntaje promedio de Silhouette alto indica un buen agrupamiento.

La función `silhouette` en el paquete de clúster se utiliza para calcular el ancho promedio de Silhouette.

> El Silhouette se puede calcular con cualquier [métrica de distancia](https://en.wikipedia.org/wiki/Distance "Distance"), como la [distancia euclidiana](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") o la [distancia Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") que discutimos en la [lección anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Nuestro puntaje es **.549**, lo que nos coloca justo en el medio. Esto indica que nuestros datos no están particularmente bien adaptados a este tipo de agrupamiento. Veamos si podemos confirmar esta sospecha de manera visual. El [paquete factoextra](https://rpkgs.datanovia.com/factoextra/index.html) proporciona funciones (`fviz_cluster()`) para visualizar agrupamientos.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


El solapamiento en los clústeres indica que nuestros datos no son particularmente adecuados para este tipo de agrupamiento, pero sigamos adelante.

## 4. Determinando el número óptimo de clústeres

Una pregunta fundamental que surge a menudo en el agrupamiento K-Means es esta: sin etiquetas de clase conocidas, ¿cómo sabes en cuántos clústeres separar tus datos?

Una forma de intentar averiguarlo es usar una muestra de datos para `crear una serie de modelos de agrupamiento` con un número creciente de clústeres (por ejemplo, de 1 a 10) y evaluar métricas de agrupamiento como el **índice de Silhouette.**

Determinemos el número óptimo de clústeres calculando el algoritmo de agrupamiento para diferentes valores de *k* y evaluando el **Suma de Cuadrados Dentro del Clúster** (WCSS, por sus siglas en inglés). La suma total de cuadrados dentro del clúster (WCSS) mide la compacidad del agrupamiento, y queremos que sea lo más pequeña posible, ya que valores más bajos significan que los puntos de datos están más cerca entre sí.

Exploremos el efecto de diferentes elecciones de `k`, desde 1 hasta 10, en este agrupamiento.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Ahora que tenemos la suma total de cuadrados dentro del grupo (tot.withinss) para cada algoritmo de agrupamiento con centro *k*, utilizamos el [método del codo](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) para encontrar el número óptimo de grupos. El método consiste en graficar la WCSS como una función del número de grupos y elegir el [codo de la curva](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") como el número de grupos a utilizar.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


El gráfico muestra una gran reducción en WCSS (lo que indica una mayor *cohesión*) a medida que el número de clústeres aumenta de uno a dos, y una reducción adicional notable de dos a tres clústeres. Después de eso, la reducción es menos pronunciada, lo que da lugar a un `codo` 💪 en el gráfico alrededor de tres clústeres. Esto es una buena indicación de que hay dos o tres clústeres de puntos de datos razonablemente bien separados.

Ahora podemos proceder a extraer el modelo de clustering donde `k = 3`:

> `pull()`: se utiliza para extraer una sola columna  
>  
> `pluck()`: se utiliza para indexar estructuras de datos como listas  


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


¡Genial! Vamos a visualizar los clústeres obtenidos. ¿Te interesa algo de interactividad usando `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Quizás hubiéramos esperado que cada clúster (representado por diferentes colores) tuviera géneros distintos (representados por diferentes formas).

Echemos un vistazo a la precisión del modelo.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


La precisión de este modelo no está mal, pero tampoco es excelente. Puede ser que los datos no se presten bien para el uso de K-Means Clustering. Estos datos están demasiado desequilibrados, tienen poca correlación y existe demasiada variabilidad entre los valores de las columnas como para agruparlos de manera efectiva. De hecho, los clústeres que se forman probablemente están muy influenciados o sesgados por las tres categorías de géneros que definimos anteriormente.

¡Aun así, fue un proceso de aprendizaje interesante!

En la documentación de Scikit-learn, puedes ver que un modelo como este, con clústeres no muy bien definidos, tiene un problema de 'varianza':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografía de Scikit-learn</figcaption>



## **Varianza**

La varianza se define como "el promedio de las diferencias al cuadrado respecto a la media" [fuente](https://www.mathsisfun.com/data/standard-deviation.html). En el contexto de este problema de clustering, se refiere a que los números de nuestro conjunto de datos tienden a divergir demasiado de la media.

✅ Este es un gran momento para pensar en todas las formas en que podrías corregir este problema. ¿Ajustar un poco más los datos? ¿Usar diferentes columnas? ¿Probar con un algoritmo distinto? Pista: Intenta [escalar tus datos](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) para normalizarlos y prueba con otras columnas.

> Prueba este '[calculador de varianza](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' para entender un poco más el concepto.

------------------------------------------------------------------------

## **🚀Desafío**

Dedica algo de tiempo a este notebook ajustando parámetros. ¿Puedes mejorar la precisión del modelo limpiando más los datos (eliminando valores atípicos, por ejemplo)? Puedes usar pesos para dar más importancia a ciertas muestras de datos. ¿Qué más podrías hacer para crear mejores clústeres?

Pista: Intenta escalar tus datos. Hay código comentado en el notebook que agrega escalado estándar para hacer que las columnas de datos se parezcan más entre sí en términos de rango. Descubrirás que, aunque el puntaje de silueta disminuye, el 'codo' en el gráfico de codo se suaviza. Esto se debe a que dejar los datos sin escalar permite que los datos con menos varianza tengan más peso. Lee un poco más sobre este problema [aquí](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Cuestionario posterior a la lección**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Revisión y Autoestudio**

-   Echa un vistazo a un simulador de K-Means [como este](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Puedes usar esta herramienta para visualizar puntos de datos de muestra y determinar sus centroides. Puedes editar la aleatoriedad de los datos, el número de clústeres y el número de centroides. ¿Te ayuda esto a tener una idea de cómo se pueden agrupar los datos?

-   También, revisa [este documento sobre K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) de Stanford.

¿Quieres probar tus recién adquiridas habilidades de clustering en conjuntos de datos que se presten bien para K-Means clustering? Consulta:

-   [Entrenar y Evaluar Modelos de Clustering](https://rpubs.com/eR_ic/clustering) usando Tidymodels y amigos

-   [Análisis de Clústeres K-Means](https://uc-r.github.io/kmeans_clustering), Guía de Programación en R para Análisis de Negocios de UC

- [Clustering K-Means con principios de datos ordenados](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tarea**

[Prueba diferentes métodos de clustering](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## AGRADECIMIENTOS A:

[Jen Looper](https://www.twitter.com/jenlooper) por crear la versión original en Python de este módulo ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) por crear las increíbles ilustraciones que hacen que R sea más accesible y atractivo. Encuentra más ilustraciones en su [galería](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Feliz aprendizaje,

[Eric](https://twitter.com/ericntay), Embajador Estudiantil Gold de Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Arte de @allison_horst</figcaption>



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
